In [1]:
## Part 3: Create a Travel Itinerary with a Corresponding Map

In [204]:
import pandas as pd
import gmaps
import gmaps.datasets
import requests
import numpy as np
from config import g_key

In [173]:
# Read Files
city_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
city_data_df.head()

,Unnamed: 0,Unnamed: 0.1,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,0,Yellowknife,CA,2020-02-23 02:36:40,62.46,-114.35,0.00,64,75,5.82,broken clouds,0.0,0.0
1,1,1,Tongliao,CN,2020-02-23 02:39:32,43.61,122.27,36.23,27,100,18.81,overcast clouds,0.0,0.0
2,2,2,Bethel,US,2020-02-23 02:38:16,41.37,-73.41,39.00,58,1,4.23,clear sky,0.0,0.0
3,3,3,Tuktoyaktuk,CA,2020-02-23 02:39:32,69.45,-133.04,1.40,84,90,3.36,light snow,0.0,0.0
4,4,4,Kaitangata,NZ,2020-02-23 02:39:32,-46.28,169.85,64.00,57,94,8.99,overcast clouds,0.0,0.0


In [195]:
# select 4 cities
city_data_df = city_data_df[city_data_df["Country"] == "US"]
city_selected_df = city_data_df.loc[(city_data_df.Lat >= 40) & (city_data_df.Lat <= 41)]
city_selected_df.head()

,Unnamed: 0,Unnamed: 0.1,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
24,24,24,West Wendover,US,2020-02-23 02:39:37,40.74,-114.07,35.6,55,1,4.70,clear sky,0.0,0.0
33,33,33,Elko,US,2020-02-23 02:35:01,41.00,-115.50,46.4,33,1,5.82,clear sky,0.0,0.0
48,48,48,Fortuna,US,2020-02-23 02:39:42,40.60,-124.16,50.0,87,90,5.82,overcast clouds,0.0,0.0
296,296,296,Craig,US,2020-02-23 02:40:38,40.52,-107.55,30.2,73,1,4.52,clear sky,0.0,0.0


In [190]:
# Latitude-longitude pairs
WestWendover = (40.74, -114.07)
Elko = (41.00, -115.50)
Fortuna = (40.60, -124.16)
Craig = (40.52, -107.55)

In [217]:
# Draw direction from 4 points
gmaps.configure(api_key = g_key)
fig = gmaps.figure()
Craig_Elko = gmaps.directions_layer(Craig, Fortuna, waypoints=[WestWendover,Elko], travel_mode='DRIVING')

fig.add_layer(Craig_Elko)
fig

Figure(layout=FigureLayout(height='420px'))

In [208]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_selected_df.copy()[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

In [210]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.


In [211]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [212]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))